In [13]:
# Ladataan apufunktiot suoraan serveriltä
%run /home/jovyan/data_loading_tools.ipynb
#%run /home/jovyan/obs_reader.ipynb
%run /home/jovyan/plotting_tools.ipynb

In [14]:
# Client täytyy käynnistää ekana
client = start_client()

In [15]:
# Alustetaan datan hakua, haetaan tunnin sadekertymä
parameters = ['Precipitation1h']
time_range = 10                   # Määrittää kuinka monelta päivältä dataa haetaan tästä hetkestä eteenpäin
model = 'EC'

delayed_data_ec = get_model_data_delay(parameters, time_range, model)
EC_params = merge_model_delayed(client, delayed_data_ec)

Done!


In [16]:
EC_params.head().compute()

<xarray.Dataset>
Dimensions:                   (time: 5, y: 5, x: 5, time_h: 5, time_bounds: 2)
Coordinates:
  * time                      (time) datetime64[ns] 2024-09-11 ... 2024-09-11...
  * y                         (y) float32 -3.883e+06 -3.876e+06 ... -3.854e+06
  * x                         (x) float32 -1.01e+06 -1.003e+06 ... -9.806e+05
  * time_h                    (time_h) datetime64[ns] 2024-09-11 ... 2024-09-...
    lat                       (y, x) float32 52.7 52.72 52.73 ... 53.0 53.01
    lon                       (y, x) float32 5.421 5.523 5.625 ... 5.621 5.724
Dimensions without coordinates: time_bounds
Data variables:
    crs                       int16 -32767
    time_bounds_h             (time_h, time_bounds) datetime64[ns] 2024-09-10...
    precipitation_amount_353  (time_h, y, x) float32 0.0003744 ... 0.0002364
Attributes:
    Conventions:  CF-1.6
    title:        <title>
    institution:  fmi.fi
    source:       <producer>

In [17]:
# # Otetaan data ulos xarrayna, koska vain yksi käyttäjä DASK-järjestelmässä ja serverin käyttö hankaloittaa tekemistä porukalla
rp1h = EC_params['precipitation_amount_353'].compute()
lat = EC_params['lat'].compute()
lon = EC_params['lon'].compute()
time = EC_params['time'].compute()
time_h = EC_params['time_h'].compute()

In [18]:
# Satunnaista testailua lokaalilla

rp1h_max = (rp1h.max(dim="time_h").compute())
print(rp1h_max.values)
rp1h_hour = (rp1h.groupby("time_h.hour").mean().compute())
print(rp1h_hour.values)
rp1h_days = (rp1h.resample(time_h="1D"))
print(rp1h_days)

print(rp1h.shape)


[[1.0861112e-03 1.3897221e-03 2.3666667e-03 ... 2.7777779e-07
  0.0000000e+00 0.0000000e+00]
 [1.2558333e-03 1.4855555e-03 1.0105555e-03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.5808333e-03 2.4247223e-03 2.6469445e-03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [4.4805557e-04 4.2222222e-04 4.9111113e-04 ... 4.0611112e-04
  4.0305554e-04 3.6444445e-04]
 [4.5916668e-04 4.3305557e-04 4.9333333e-04 ... 4.0416667e-04
  4.1499999e-04 3.8472223e-04]
 [4.5277778e-04 4.6333333e-04 4.8361113e-04 ... 4.1222220e-04
  4.2861109e-04 4.1638888e-04]]
[[[3.40404076e-05 4.04797975e-05 5.34090905e-05 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [3.40151528e-05 3.61363636e-05 3.92171714e-05 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [2.90909084e-05 2.82575766e-05 3.48989888e-05 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  ...
  [4.99242451e-05 4.71717212e-05 4.51767664e-05 ... 1.80050502e-05
   1.93939395e-05 1.99494953e-05]
  [4.52020213e-05 4.684343

# Meteorologit voi aloittaa tekemisen tästä kohtaa. Ylimpänä oleva "solu" pitää ajaa, jotta saadaan metodit käyttöön

In [19]:
# Lasketaan vuorokauden sadesumma

max_rp1h = xr.full_like(rp1h,0)         #kopio alk.per arraysta, johon voidaan tallentaa aina seuraavan 6h maksimit. Array sisältää pelkkää nollaa
for x in range(len(rp1h.time_h)):
    end = x+23      #seuraava 24h
    slice = rp1h[x:end,:,:]     #seuraava 24h slice
    rp1h_max = slice.sum(dim='time_h') #otetaan siitä maksimi
    max_rp1h[x,:,:] = rp1h_max.data #tallennetaan maksimit oikeaan paikkaan datassa
time_max = max_rp1h['time_h'].compute()
print(x)
print((rp1h.shape))


258
(259, 280, 255)


# Tästä eteenpäin tehdään visualisointeja

In [24]:
# Plotataan lokaali muuttuja
def draw_local(parameter, t): 
    # Määritellään kartta:
    ax = generate_fig_properties(lat, lon)
    #ax.figure.set_size_inches([18, 13])

    # Käsitellään dataa
    chart = max_rp1h.isel(time_h=t)
    chart = chart * 3600    # Muutetaan kg/h -> mm/h about
    chart = chart.where(chart >= 5)
    cmap = 'nipy_spectral'
    vmin = 0
    vmax = 50
    
    pcolormesh_map_plt(chart, vmin, vmax, cmap, lon, lat)

In [25]:
params = ['precipitation_amount_353']
widgets.interact(draw_local, parameter=params, t=(0, len(time_max) - 1))

interactive(children=(Dropdown(description='parameter', options=('precipitation_amount_353',), value='precipit…

<function __main__.draw_local(parameter, t)>